In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#x_train = np.load('/content/drive/My Drive/dlhw1p2/train.npy', allow_pickle=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#from google.colab import files
#df.to_csv('/content/drive/My Drive/dlhw1p2/ex.csv', index=False)
#files.download('/content/drive/My Drive/dlhw1p2/ex.csv')

In [ ]:
! pip install strsim

In [ ]:
from similarity.normalized_levenshtein import NormalizedLevenshtein
from collections import Counter

import json
import requests

import os
import numpy as np
from PIL import Image

import numpy as np
import pandas as pd
'''import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader'''
import time

from PIL import Image
import requests
from io import BytesIO

#import torchvision.models as models
#import torchvision.transforms as transforms

#from torch.autograd import Variable

# Install required libraries
#!pip install torch>=1.2.0 tensorboard future tqdm
#from torch.utils.tensorboard import SummaryWriter
#from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
url = 'http://datasets.cvc.uab.es/rrc/train_task_3.json'
r = requests.get(url)

train = r.json()
#print(r.json())

In [ ]:
len(train['data'])

26074

In [ ]:
#pretty quick
!cd "/content/drive/My Drive/Multimodal ML (11-777)/stvqa_data"
!tar -xvf "/content/drive/My Drive/Multimodal ML (11-777)/stvqa_data/stvqa_training_images.tar"

Streaming output truncated to the last 5000 lines.
imageNet/n02690373_5160.JPEG
coco-text/COCO_train2014_000000421341.jpg
imageNet/n02783161_11991.JPEG
imageNet/n07248320_101788.JPEG
imageNet/n02799071_34797.JPEG
imageNet/n03729826_4215.JPEG
VisualGenome/1/2316440.jpg
coco-text/COCO_train2014_000000127268.jpg
imageNet/n02172182_4833.JPEG
VisualGenome/1/2323169.jpg
VisualGenome/2/2415897.jpg
VisualGenome/1/2337520.jpg
VisualGenome/1/2319059.jpg
VisualGenome/1/2339483.jpg
IIIT_text/2411.jpg
coco-text/COCO_train2014_000000007220.jpg
coco-text/COCO_train2014_000000028103.jpg
imageNet/n03649909_12964.JPEG
coco-text/COCO_train2014_000000202990.jpg
VisualGenome/1/2357215.jpg
imageNet/n04591157_1476.JPEG
coco-text/COCO_train2014_000000105733.jpg
VisualGenome/1/2338634.jpg
coco-text/COCO_train2014_000000107838.jpg
VisualGenome/1/2322375.jpg
VisualGenome/1/2356027.jpg
imageNet/n02917067_18193.JPEG
coco-text/COCO_train2014_000000124914.jpg
imageNet/n02669723_468.JPEG
coco-text/COCO_train2014_0000

In [ ]:
train['data'][10]

{'answers': ['Airfrance'],
 'dataset': 'VisualGenome/2',
 'file_name': '2401851.jpg',
 'file_path': 'VisualGenome/2/2401851.jpg',
 'image_height': 334,
 'image_width': 500,
 'question': 'What airline is this?',
 'question_id': 12,
 'question_tokens': ['What', 'airline', 'is', 'this', '?'],
 'set_name': 'train'}

In [ ]:
train_data = None
with open("/content/drive/My Drive/Multimodal ML (11-777)/stvqa_data/train_task_3.json", 'r') as file:
  train_data = json.load(file)

In [ ]:
training_df = pd.DataFrame(train_data['data'])

In [ ]:
training_df.head()

,set_name,file_name,question,image_width,answers,dataset,question_tokens,image_height,file_path,question_id
0,train,COCO_train2014_000000347021.jpg,What is the book authors first name?,256,[Susan],coco-text,"[What, is, the, book, authors, first, name, ?]",256,coco-text/COCO_train2014_000000347021.jpg,0
1,train,COCO_train2014_000000347021.jpg,what is the book authors last name?,256,[Blackmore],coco-text,"[what, is, the, book, authors, last, name, ?]",256,coco-text/COCO_train2014_000000347021.jpg,1
2,train,COCO_train2014_000000334557.jpg,Who's name is on the cake?,256,[Chris],coco-text,"[Who, 's, name, is, on, the, cake, ?]",256,coco-text/COCO_train2014_000000334557.jpg,2
3,train,2341689.jpg,What does the writing on the bottom of the pic...,433,[Big Kites],VisualGenome/1,"[What, does, the, writing, on, the, bottom, of...",500,VisualGenome/1/2341689.jpg,4
4,train,2359981.jpg,What company is advertised on the red sign?,500,[Helly Hansen],VisualGenome/1,"[What, company, is, advertised, on, the, red, ...",333,VisualGenome/1/2359981.jpg,5


In [ ]:
len(training_df)

26074

In [ ]:
predictions_df = pd.read_json("/content/drive/My Drive/Colab Notebooks/fb_mmf/default_stvqa_testvqa_val_predictions/stvqa_run_val_hierarchical_attention.json")

In [ ]:
predictions_df.head()

,question_id,image_id,answer,pred_source
0,13885,10011288,1,[VOCAB]
1,23101,10018887,24,[OCR]
2,16784,10013686,gp-1652,[OCR]
3,7129,10005783,stay hungry,"[OCR, OCR]"
4,916,10000747,earthshots.org,[OCR]


We can treat question ids as the unique identifier of the dataset!

In [ ]:
len(set(predictions_df['question_id'])) #length of validation dataset

2628

In [ ]:
len(set(training_df['question_id']))

26074

In [ ]:
#make sure all our predictions can be checked with the answers in the training set (we want this length to be 0)
set(predictions_df['question_id']).difference(set(training_df['question_id']))

set()

In [ ]:
#training_df.set_index('question_id', inplace = True)
#training_df.head()

In [ ]:
#predictions_df.set_index('question_id', inplace = True)
#predictions_df.head()

In [ ]:
training_df[training_df['question_id'] == 5422]

,set_name,file_name,question,image_width,answers,dataset,question_tokens,image_height,file_path,question_id
4371,train,COCO_train2014_000000110217.jpg,what word is on the red sign?,256,[stop],coco-text,"[what, word, is, on, the, red, sign, ?]",256,coco-text/COCO_train2014_000000110217.jpg,5422


In [ ]:
x, y = [], []
for i in range(5):
  pred_values = predictions_df.iloc[i]
  question_id = pred_values['question_id']
  actual_values = training_df[training_df['question_id'] == question_id]
  print(pred_values['answer'])
  x.append(pred_values['answer'])
  print(actual_values['answers'].item())
  y.append(actual_values['answers'].item())

1
['6']
24
['3']
gp-1652
['GP-1652', 'GP 1652']
stay hungry
['Stay hungry stay foolish', '"STAY HUNGRY, STAY FOOLISH."']
earthshots.org
['EarthShots.org']


STVQA Evaluation Metric

In [ ]:
normalized_levenshtein = NormalizedLevenshtein()
print(normalized_levenshtein.distance('red up', 'up'))

0.6666666666666666


In [ ]:
#stvqa can have up to two answers (1 is correct; 0 is wrong; sliding scale) (paper never says lowercase)
metrics = []
for i in range(len(x)):
  prediction = x[i] #x[i].lower()
  labels = y[i]
  max = 0

  for label in labels:
    label = label #label.lower()
    nls = normalized_levenshtein.distance(prediction, label)
    curr_score = 0
    if nls >= 0.5:
      curr_score = 0
    else:
      curr_score = 1 - nls
    
    if curr_score >= max:
      max = curr_score
    
  metrics.append(max)

In [ ]:
anls = float(sum(metrics))/len(metrics)

In [ ]:
anls

0.31428571428571433

In [ ]:
metrics

[0, 0, 0.7142857142857143, 0, 0.8571428571428572]

### ACTUAL STUFF

In [ ]:
temp_df = training_df[training_df['question_id'] == 5422].set_index('question_id')
#temp_df['answer'], temp_df['pred_source'] = [np.nan, 'dogs', 3]

In [ ]:
combo_df = pd.DataFrame()
#range of len(predictions_df)
for i in range(len(predictions_df)):
  pred_values = predictions_df.iloc[i]
  question_id = pred_values['question_id']
  temp_df = training_df[training_df['question_id'] == question_id].set_index('question_id')
  #print(len(temp_df))
  #print(pred_values)
  #print(temp_df)
  temp_df['answer'] = pred_values['answer']
  temp_df['pred_source'] = [pred_values['pred_source']] #list
  combo_df = pd.concat([combo_df, temp_df], axis=0)

In [ ]:
combo_df

,set_name,file_name,question,image_width,answers,dataset,question_tokens,image_height,file_path,answer,pred_source
question_id,,,,,,,,,,,
13885,train,n03445924_3267.JPEG,What is the number on the first cart?,500,[6],imageNet,"[What, is, the, number, on, the, first, cart, ?]",375,imageNet/n03445924_3267.JPEG,1,[VOCAB]
23101,train,2415074.jpg,What is the number next to the word FINCHLEY?,500,[3],VisualGenome/2,"[What, is, the, number, next, to, the, word, F...",375,VisualGenome/2/2415074.jpg,24,[OCR]
16784,train,n04146614_2155.JPEG,What is the license plate of the blue bus?,679,"[GP-1652, GP 1652]",imageNet,"[What, is, the, license, plate, of, the, blue,...",450,imageNet/n04146614_2155.JPEG,gp-1652,[OCR]
7129,train,COCO_train2014_000000548702.jpg,What quot is written in the picture?,256,"[Stay hungry stay foolish, ""STAY HUNGRY, STAY ...",coco-text,"[What, quot, is, written, in, the, picture, ?]",256,coco-text/COCO_train2014_000000548702.jpg,stay hungry,"[OCR, OCR]"
916,train,n02487347_3680.JPEG,What web address is located at the bottom?,600,[EarthShots.org],imageNet,"[What, web, address, is, located, at, the, bot...",400,imageNet/n02487347_3680.JPEG,earthshots.org,[OCR]
...,...,...,...,...,...,...,...,...,...,...,...
31501,train,COCO_train2014_000000525555.jpg,What four letters are shown on the red umbrella?,256,[LHJD],coco-text,"[What, four, letters, are, shown, on, the, red...",256,coco-text/COCO_train2014_000000525555.jpg,versaci,[VOCAB]
14397,train,COCO_train2014_000000268159.jpg,what number is visible on the sign in the uppe...,256,[5],coco-text,"[what, number, is, visible, on, the, sign, in,...",256,coco-text/COCO_train2014_000000268159.jpg,10,[VOCAB]
21832,train,COCO_train2014_000000568116.jpg,What is the number on the bottle?,256,[780478],coco-text,"[What, is, the, number, on, the, bottle, ?]",256,coco-text/COCO_train2014_000000568116.jpg,six,[VOCAB]


In [ ]:
#stvqa can have up to two answers (1 is correct; 0 is wrong; sliding scale) (paper never says lowercase)
metrics = []
all_predictions = list(combo_df['answer'])
all_labels = list(combo_df['answers'])

for i in range(len(predictions_df)):
  prediction = all_predictions[i].lower() #one value
  labels = all_labels[i] #list
  max = 0

  for label in labels:
    label = label.lower()
    nls = normalized_levenshtein.distance(prediction, label)
    curr_score = 0
    if nls >= 0.5:
      curr_score = 0
    else:
      curr_score = 1 - nls
    
    if curr_score >= max:
      max = curr_score
    
  metrics.append(max)

In [ ]:
metrics[20:30]

[0, 0, 0, 1.0, 1.0, 0, 1.0, 0, 0.5454545454545454, 0.7222222222222222]

In [ ]:
np.mean(metrics)

0.46533106377891026

In [ ]:
combo_df['anls'] = metrics 

In [ ]:
combo_df

,set_name,file_name,question,image_width,answers,dataset,question_tokens,image_height,file_path,answer,pred_source,anls
question_id,,,,,,,,,,,,
13885,train,n03445924_3267.JPEG,What is the number on the first cart?,500,[6],imageNet,"[What, is, the, number, on, the, first, cart, ?]",375,imageNet/n03445924_3267.JPEG,1,[VOCAB],0.000000
23101,train,2415074.jpg,What is the number next to the word FINCHLEY?,500,[3],VisualGenome/2,"[What, is, the, number, next, to, the, word, F...",375,VisualGenome/2/2415074.jpg,24,[OCR],0.000000
16784,train,n04146614_2155.JPEG,What is the license plate of the blue bus?,679,"[GP-1652, GP 1652]",imageNet,"[What, is, the, license, plate, of, the, blue,...",450,imageNet/n04146614_2155.JPEG,gp-1652,[OCR],1.000000
7129,train,COCO_train2014_000000548702.jpg,What quot is written in the picture?,256,"[Stay hungry stay foolish, ""STAY HUNGRY, STAY ...",coco-text,"[What, quot, is, written, in, the, picture, ?]",256,coco-text/COCO_train2014_000000548702.jpg,stay hungry,"[OCR, OCR]",0.000000
916,train,n02487347_3680.JPEG,What web address is located at the bottom?,600,[EarthShots.org],imageNet,"[What, web, address, is, located, at, the, bot...",400,imageNet/n02487347_3680.JPEG,earthshots.org,[OCR],1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
31501,train,COCO_train2014_000000525555.jpg,What four letters are shown on the red umbrella?,256,[LHJD],coco-text,"[What, four, letters, are, shown, on, the, red...",256,coco-text/COCO_train2014_000000525555.jpg,versaci,[VOCAB],0.000000
14397,train,COCO_train2014_000000268159.jpg,what number is visible on the sign in the uppe...,256,[5],coco-text,"[what, number, is, visible, on, the, sign, in,...",256,coco-text/COCO_train2014_000000268159.jpg,10,[VOCAB],0.000000
21832,train,COCO_train2014_000000568116.jpg,What is the number on the bottle?,256,[780478],coco-text,"[What, is, the, number, on, the, bottle, ?]",256,coco-text/COCO_train2014_000000568116.jpg,six,[VOCAB],0.000000


## ERROR ANALYSIS

I'm setting:

wrong: threshold <= 0.5 (technically they all go to 0)

right: threshold > 0.5

In [ ]:
right = combo_df[combo_df['anls'] > 0.5]
wrong = combo_df[combo_df['anls'] <= 0.5]

In [ ]:
#ACCURACY
len(right) / len(combo_df)

0.5003805175038052

In [ ]:
np.mean(combo_df['anls'])

0.46533106377891

In [ ]:
wrong

,set_name,file_name,question,image_width,answers,dataset,question_tokens,image_height,file_path,answer,pred_source,anls
question_id,,,,,,,,,,,,
13885,train,n03445924_3267.JPEG,What is the number on the first cart?,500,[6],imageNet,"[What, is, the, number, on, the, first, cart, ?]",375,imageNet/n03445924_3267.JPEG,1,[VOCAB],0.0
23101,train,2415074.jpg,What is the number next to the word FINCHLEY?,500,[3],VisualGenome/2,"[What, is, the, number, next, to, the, word, F...",375,VisualGenome/2/2415074.jpg,24,[OCR],0.0
7129,train,COCO_train2014_000000548702.jpg,What quot is written in the picture?,256,"[Stay hungry stay foolish, ""STAY HUNGRY, STAY ...",coco-text,"[What, quot, is, written, in, the, picture, ?]",256,coco-text/COCO_train2014_000000548702.jpg,stay hungry,"[OCR, OCR]",0.0
18692,train,COCO_train2014_000000342988.jpg,What is the time on the microwave?,256,"[1002, 10:02]",coco-text,"[What, is, the, time, on, the, microwave, ?]",256,coco-text/COCO_train2014_000000342988.jpg,3:18,[VOCAB],0.0
5098,train,COCO_train2014_000000273951.jpg,What brand of skis is the man wearing his sung...,256,"[fischer fischer, fischer]",coco-text,"[What, brand, of, skis, is, the, man, wearing,...",256,coco-text/COCO_train2014_000000273951.jpg,head,[VOCAB],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13485,train,n02092002_10292.JPEG,what website do i go to?,482,"[www.bj-photography.co.uk, ww.bj.photography.c...",imageNet,"[what, website, do, i, go, to, ?]",504,imageNet/n02092002_10292.JPEG,photo,[VOCAB],0.0
31501,train,COCO_train2014_000000525555.jpg,What four letters are shown on the red umbrella?,256,[LHJD],coco-text,"[What, four, letters, are, shown, on, the, red...",256,coco-text/COCO_train2014_000000525555.jpg,versaci,[VOCAB],0.0
14397,train,COCO_train2014_000000268159.jpg,what number is visible on the sign in the uppe...,256,[5],coco-text,"[what, number, is, visible, on, the, sign, in,...",256,coco-text/COCO_train2014_000000268159.jpg,10,[VOCAB],0.0


In [ ]:
for item in list(wrong['question']):
  if 'jersey' in item:
    print(item)

What number is the kid in the orange jersey?
What team name is on the black jersey?
What is this player's jersey number?
What number is the player in the red jersey?
What number is printed on the jersey of the batter?
What is the number on the jersey?
What number is on the batter's jersey?
What team is playing in a red jersey?
What is the name on #51's jersey?
What number is the kid in the black jersey


In [110]:
right[right['question'] == 'What are the visible letters on the front of the jersey?']

,set_name,file_name,question,image_width,answers,dataset,question_tokens,image_height,file_path,answer,pred_source,anls
question_id,,,,,,,,,,,,
23470,train,61551.jpg,What are the visible letters on the front of t...,1024,[GELS],VisualGenome/1,"[What, are, the, visible, letters, on, the, fr...",768,VisualGenome/1/61551.jpg,geils,[OCR],0.8


In [ ]:
#seems most of incorrectly answered questions did have unambiguous answer
Counter([len(ans) for ans in wrong['answers']])

Counter({1: 1112, 2: 201})

In [ ]:
#same as distribution above roughly
Counter([len(ans) for ans in right['answers']])

Counter({1: 1168, 2: 147})

In [ ]:
#evenly from OCR versus vocab in incorrect
Counter([source for pred_sources in wrong['pred_source'] for source in pred_sources])

Counter({'OCR': 823, 'VOCAB': 967})

In [ ]:
#more from OCR in correct
Counter([source for pred_sources in right['pred_source'] for source in pred_sources])

Counter({'OCR': 1433, 'VOCAB': 366})

In [ ]:
#which dataset most of wrong came from
Counter(wrong['dataset'])

Counter({'IIIT_text': 51,
         'VisualGenome/1': 306,
         'VisualGenome/2': 87,
         'coco-text': 530,
         'icdar': 55,
         'imageNet': 216,
         'vizwiz': 68})

In [ ]:
Counter(combo_df['dataset'])

Counter({'IIIT_text': 141,
         'VisualGenome/1': 695,
         'VisualGenome/2': 212,
         'coco-text': 926,
         'icdar': 110,
         'imageNet': 442,
         'vizwiz': 102})

In [ ]:
#question tokens
Counter([tok for q_tokens in wrong['question_tokens'] for tok in q_tokens]).most_common(30)

[('the', 1598),
 ('?', 1234),
 ('What', 972),
 ('is', 895),
 ('of', 461),
 ('on', 438),
 ('in', 205),
 ('name', 178),
 ('this', 146),
 ('number', 137),
 ('sign', 122),
 ('are', 118),
 ('does', 109),
 ('what', 100),
 ('written', 98),
 ('word', 90),
 ('to', 70),
 ('bus', 65),
 ('brand', 63),
 ('say', 60),
 ('How', 58),
 ('red', 49),
 ('right', 49),
 ("'s", 49),
 ('white', 47),
 ('left', 47),
 ('many', 45),
 ('top', 43),
 ('first', 42),
 ('type', 42)]

In [ ]:
Counter([tok for q_tokens in right['question_tokens'] for tok in q_tokens]).most_common(30)

[('the', 1672),
 ('?', 1226),
 ('What', 1063),
 ('is', 953),
 ('on', 456),
 ('of', 413),
 ('name', 219),
 ('sign', 199),
 ('in', 171),
 ('this', 160),
 ('number', 154),
 ('does', 147),
 ('written', 137),
 ('word', 112),
 ('say', 102),
 ('bus', 94),
 ('what', 93),
 ('are', 74),
 ('red', 68),
 ('brand', 60),
 ('picture', 57),
 ('street', 57),
 ('at', 53),
 ('top', 53),
 ('bottom', 52),
 ("'s", 51),
 ('company', 49),
 ('photo', 48),
 ('train', 47),
 ('year', 46)]

In [ ]:
#identifying number of objects is more difficult; or concepts like colors of objects
Counter([subitem for item in wrong['answers'] for subitem in item]).most_common(30)

[('3', 11),
 ('6', 10),
 ('2', 10),
 ('7', 9),
 ('4', 7),
 ('1', 7),
 ('5', 6),
 ('25', 5),
 ('10', 5),
 ('{}', 5),
 ('15', 4),
 ('24', 4),
 ('A', 4),
 ('Two', 4),
 ('sneaktr', 3),
 ('41', 3),
 ('B', 3),
 ('0', 3),
 ('17', 3),
 ('Brown', 3),
 ('RED', 3),
 ('Stop', 3),
 ('Three', 3),
 ('Blue', 3),
 ('USA', 3),
 ('One Way', 2),
 ('c', 2),
 ('8', 2),
 ('AC', 2),
 ('E', 2)]

In [ ]:
#reading directions or signs is easier
Counter([subitem for item in right['answers'] for subitem in item]).most_common(30)

[('STOP', 22),
 ('Stop', 12),
 ('stop', 12),
 ('2011', 10),
 ('2', 9),
 ('2007', 8),
 ('Tennis', 7),
 ('15', 6),
 ('2013', 6),
 ('2012', 6),
 ('2009', 5),
 ('White', 4),
 ('Delta', 4),
 ('2008', 4),
 ('IndiGo', 4),
 ('Samsung', 4),
 ('Kia', 4),
 ('11', 4),
 ('12', 3),
 ('KLM', 3),
 ('30', 3),
 ('2005', 3),
 ('6', 3),
 ('10', 3),
 ('101', 3),
 ('Red', 3),
 ('Police', 3),
 ('POLICE', 3),
 ('2010', 3),
 ('9', 3)]

In [ ]:
Counter([item for item in wrong['answer']]).most_common(30)

[('2', 19),
 ('5', 13),
 ('8', 12),
 ('3', 12),
 ('1', 11),
 ('stop', 11),
 ('coca cola', 11),
 ('red', 9),
 ('7', 9),
 ('4', 8),
 ('blue', 8),
 ('a', 6),
 ('air', 6),
 ('{}', 6),
 ('white', 6),
 ('three', 6),
 ('one', 5),
 ('exit', 5),
 ('50', 5),
 ('13', 5),
 ('sony', 5),
 ('6', 4),
 ('29', 4),
 ('w', 4),
 ('alibaba.com.cn', 4),
 ('$2.00', 4),
 ('ford', 4),
 ('art', 4),
 ('e', 4),
 ('75', 4)]

In [ ]:
Counter([item for item in right['answer']]).most_common(30)

[('stop', 45),
 ('2', 9),
 ('2007', 9),
 ('2013', 8),
 ('one way', 7),
 ('2012', 7),
 ('tennis', 7),
 ('2011', 7),
 ('delta', 6),
 ('coca cola', 6),
 ('2009', 6),
 ('police', 6),
 ('red', 5),
 ('school bus', 5),
 ('indigo', 5),
 ('kia', 5),
 ('honda', 5),
 ('15', 4),
 ('white', 4),
 ('nokia', 4),
 ('10', 4),
 ('hollywood', 4),
 ('11', 4),
 ('rolex', 4),
 ('exit', 4),
 ('12', 3),
 ('klm', 3),
 ('london', 3),
 ('birthday', 3),
 ('2005', 3)]

In [ ]:
#Image.open('path')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
corpus = [" ".join(item).strip() for item in right['question_tokens']]

In [ ]:
X = vectorizer.fit_transform(corpus)

In [ ]:
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
top_features

['zamperl',
 'ingredient',
 'info',
 'indicating',
 'inbetween',
 'if',
 'identification',
 'id',
 'ichird',
 'hydrant',
 'hotel',
 'hosting',
 'hood',
 'honking',
 'holiday',
 'hitchhiker',
 'highway',
 'highgate',
 'high',
 'information']